<a href="https://colab.research.google.com/github/Kevin2558/Data_Science/blob/main/02_Preprocesamiento/Desbalanceo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Datos desbalanceados**

En tareas de clasificación, si una clase aparece mucho más que otra (por ejemplo, 95% clase 0 y 5% clase 1), muchos modelos simplemente predicen la clase mayoritaria.

Esto genera falsos positivos o falsos negativos críticos, especialmente en:

- Detección de fraude
- Diagnóstico médico
- Riesgo financiero

La solución a esto es usar técnicas como SMOTE, SMOTETomek, ADASYN, RandomUnderSampler, etc. antes del entrenamiento del modelo.

Veamos un ejemplo comparativo para utilizar los samplers.

**Sin balancear (baseline)**
- Entrenamos el modelo tal cual, sin hacer ningún ajuste.
- Puede tener alta accuracy, pero pésimo recall para la clase minoritaria.

**SMOTE**
- Genera ejemplos sintéticos de la clase minoritaria interpolando entre vecinos.
- Ayuda cuando hay pocos datos minoritarios y no queremos perder datos de la clase mayoritaria.

**RandomUnderSampler**
- Elimina aleatoriamente muestras de la clase mayoritaria.
- Es útil cuando se prefiere un dataset más pequeño y balanceado, pero puede perder información.

**SMOTETomek**
- Combina SMOTE con Tomek Links, que remueve puntos ambiguos entre clases (cerca de la frontera).
- Mejora la separación de clases y reduce ruido.

**ADASYN (Adaptive Synthetic Sampling)**
- Variante de SMOTE que genera más ejemplos sintéticos donde la clase minoritaria es más difícil de aprender (zonas de frontera).
- Focaliza el oversampling.



In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline as ImbPipeline

# Samplers
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

# Dataset simulado desbalanceado
X, y = make_classification(
    n_samples=1000, n_features=10,
    weights=[0.9, 0.1], random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Lista de técnicas a evaluar
samplers = {
    "Baseline (Sin balancear)": None,
    "SMOTE": SMOTE(random_state=42),
    "RandomUnderSampler": RandomUnderSampler(random_state=42),
    "SMOTETomek": SMOTETomek(random_state=42),
    "ADASYN": ADASYN(random_state=42)
}

for nombre, sampler in samplers.items():
    print(f"\n{'='*30}\n{nombre}\n{'='*30}")

    if sampler is None:
        # Sin balanceo
        pipeline = ImbPipeline([
            ('scaler', StandardScaler()),
            ('model', LogisticRegression())
        ])
    else:
        # Con técnica de balanceo
        pipeline = ImbPipeline([
            ('sampler', sampler),
            ('scaler', StandardScaler()),
            ('model', LogisticRegression())
        ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print(classification_report(y_test, y_pred, digits=3))


Baseline (Sin balancear)
              precision    recall  f1-score   support

           0      0.940     0.978     0.958       224
           1      0.706     0.462     0.558        26

    accuracy                          0.924       250
   macro avg      0.823     0.720     0.758       250
weighted avg      0.916     0.924     0.917       250


SMOTE
              precision    recall  f1-score   support

           0      0.976     0.897     0.935       224
           1      0.477     0.808     0.600        26

    accuracy                          0.888       250
   macro avg      0.727     0.853     0.767       250
weighted avg      0.924     0.888     0.900       250


RandomUnderSampler
              precision    recall  f1-score   support

           0      0.980     0.875     0.925       224
           1      0.440     0.846     0.579        26

    accuracy                          0.872       250
   macro avg      0.710     0.861     0.752       250
weighted avg      0.9

## Conclusiones de los resultados

Observando las métricas de `precision`, `recall` y `f1-score` para la clase minoritaria (clase 1) en cada técnica de balanceo, podemos sacar las siguientes conclusiones:

*   **Baseline (Sin balancear):** La precisión es alta (0.706), lo que significa que cuando predice la clase 1, es bastante acertado. Sin embargo, el recall es bajo (0.462), indicando que solo identifica correctamente un pequeño porcentaje de los casos reales de la clase 1. El f1-score (0.558) es moderado. Esto es típico en datasets desbalanceados, donde el modelo tiende a predecir la clase mayoritaria.

*   **SMOTE:** SMOTE mejora significativamente el recall para la clase 1 (0.808) en comparación con la baseline, lo que significa que identifica una mayor proporción de los casos reales de la clase 1. Sin embargo, la precisión para la clase 1 disminuye (0.477), lo que indica que cuando predice la clase 1, hay más falsos positivos. El f1-score (0.600) es mejor que la baseline.

*   **RandomUnderSampler:** Similar a SMOTE, RandomUnderSampler también mejora el recall para la clase 1 (0.846) de manera considerable. La precisión para la clase 1 (0.440) es la más baja entre las técnicas de balanceo, resultando en más falsos positivos. El f1-score (0.579) es comparable al de SMOTE.

*   **SMOTETomek:** Esta técnica combinada muestra un recall para la clase 1 (0.808) similar a SMOTE. La precisión para la clase 1 (0.467) es ligeramente menor que SMOTE. El f1-score (0.592) es muy similar al de SMOTE.

*   **ADASYN:** ADASYN logra el recall más alto para la clase 1 (0.885), identificando la mayor proporción de casos reales de la clase 1. No obstante, tiene la precisión más baja para la clase 1 (0.411), lo que resulta en el mayor número de falsos positivos. El f1-score (0.561) es similar a la baseline.

**En resumen:**

Las técnicas de balanceo (SMOTE, RandomUnderSampler, SMOTETomek, ADASYN) logran mejorar significativamente el **recall** para la clase minoritaria (clase 1) en comparación con el modelo sin balancear. Esto es crucial en escenarios donde es importante identificar la mayoría de los casos positivos (por ejemplo, detectar enfermedades o fraude).

Sin embargo, esta mejora en el recall generalmente viene con una disminución en la **precisión** para la clase 1, lo que significa que hay un aumento en los falsos positivos.

La elección de la mejor técnica dependerá del problema específico y de la métrica que se priorice. Si minimizar los falsos negativos es más importante, técnicas como ADASYN o RandomUnderSampler podrían ser preferibles (aunque a costa de más falsos positivos). Si se busca un equilibrio entre precisión y recall, SMOTE o SMOTETomek podrían ser buenas opciones.